In [1]:
import transformers
import torch
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, TrainingArguments, GPT2Config, Trainer
model_path = '/mntcephfs/data/ruoyusun/common_dirs/gpt2-small'
tokenizer = AutoTokenizer.from_pretrained(model_path)


/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905969073/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [16]:
dataset = load_dataset('/mntcephfs/data/ruoyusun/hanyizhou/wikitext-103/dataset/Salesforce_wikitext')
dataset = dataset['train']
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [10]:
def precess(examples):
    context = [e+ tokenizer.eos_token for e in examples['text']]
    return tokenizer(context, max_length=512, truncation=True)

In [12]:
data_wikitext = dataset.map(precess,remove_columns=dataset.column_names,batched=True)
data_wikitext.save_to_disk('/mntcephfs/data/ruoyusun/hanyizhou/data/wikitext')

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1801350 [00:00<?, ? examples/s]

In [27]:
from gpt2_modified import GPT2_FireWithCoPE,GPT2_FIRE,GPT2_CoPE
model_mix = GPT2_FireWithCoPE.from_pretrained('./output/fire_with_cope/checkpoint-41250')
model_fire = GPT2_FIRE.from_pretrained('./output/fire/checkpoint-41250')
model_cope = GPT2_CoPE.from_pretrained('./output/cope/checkpoint-41250', n_posmax=384)
output_mix = model_mix(input_ids = torch.tensor(ncoding['input_ids']), labels = torch.tensor(ncoding['input_ids']))
output_fire = model_fire(input_ids = torch.tensor(ncoding['input_ids']), labels = torch.tensor(ncoding['input_ids']))
output_cope = model_cope(input_ids = torch.tensor(ncoding['input_ids']), labels = torch.tensor(ncoding['input_ids']))
output_mix.loss, output_fire.loss, output_cope.loss

(tensor(3.5734, grad_fn=<NllLossBackward0>),
 tensor(3.2478, grad_fn=<NllLossBackward0>),
 tensor(3.5918, grad_fn=<NllLossBackward0>))

In [5]:
batch_size = 256
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
trainingset = DataLoader(dataset, batch_size= batch_size, collate_fn= DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                                                                      mlm=False))

In [11]:
next(enumerate(trainingset))[1]['input_ids'][0]

tensor([50256, 44140,  2741, 32083,  5016, 20879,  8474,   287,  4544,  2852,
           64,     0,   198,  5211,   345,   765,   284,   651,  1365,   379,
         1642, 12625, 32083,    30,   921,   481,   423,   262,  3663,    11,
         1234,   428,   319,   534, 11845,   783,    13,  3635,    11,  2693,
         2534,   358,  4654,  2159,  5016, 32083, 15869,    11,  8832,  8528,
          323,   422,   406, 19129,   283, 25416, 13804,    13,   679,   481,
          307,  7743,   257, 31516,  1241,  1398,   329,  2506,   508,  3382,
          284,   651,  1365,   351,   511, 35956,  4678,    13,   198,  1544,
          481,  4545,   345,  2279,   345,   761,   284,   760,   284,  9320,
          287,   257,   509, 22923, 32083,  5449,    11,  1390,  7605,    11,
        14296,    11, 46881,    11,  6174,  6356,   290, 15797,  2229,    11,
         5556, 41644,   290,  2046,  1321,    13,   198,   464,  1575,   284,
          307,   287,   262,  1398,   318,   720,  2327,   583, 

In [5]:
from gpt2_modified import GPT2_CoPE, GPT2_FIRE
config = GPT2Config()
config.n_positions = 1024
config.resid_pdrop = 0.0
config.embd_pdrop = 0.0
config.attn_pdrop = 0.0
config.vocab_size = 50258
model_cope = GPT2_CoPE(config,n_posmax=128)
model_fire = GPT2_FIRE(config)

In [8]:
args_cope = TrainingArguments(
    output_dir='./output/cope',
    per_device_train_batch_size=16,
    learning_rate= 6e-4,
    num_train_epochs=300,
    logging_steps = 128
)
args_fire = TrainingArguments(
    output_dir='./output/fire',
    per_device_train_batch_size=16,
    learning_rate= 6e-4,
    num_train_epochs=300,
    logging_steps = 128
)

In [9]:
trainer_cope = Trainer(
    args = args_cope,
    model = model_cope,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset = dataset                                                                                   
)

/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer_cope.train()

In [ ]:
#注意, 在接下来的amp部分我要用batched这个参数, 所以我这里的输入是一个batch的example
def preprocess_function(example):
    content = [e+tokenizer.eos_token for e in example['text']]
    return tokenizer(content,max_length=2048, truncation=True)


In [ ]:
clean_dataset = dataset.map(preprocess_function, batched=True)